Import necessary library

In [42]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from datetime import datetime, timedelta
import pytz
import pickle
import os

API_KEY = '

In [43]:
API_KEY = 'c5385722edf9ea23965d802ab9842e35'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

Fetch the current weather data

In [44]:
def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()

    return {
        'city': data['name'],
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'temp_max': round(data['main']['temp_max']),
        'humidity': round(data['main']['humidity']),
        'description': data['weather'][0]['description'],
        'country': data['sys']['country'],
        'wind_gust_dir': data['wind']['deg'],
        'Wind_Gust_Speed': data['wind']['speed'],
        'pressure' : data['main']['pressure']
    }

Read the data

In [45]:
def read_historical_data(filename):
  df=pd.read_csv(filename)
  df=df.dropna()
  df=df.drop_duplicates()
  return df

prepare the model for training

In [46]:
def prepare_data(data):
  le=LabelEncoder()
  data['WindGustDir']=le.fit_transform(data['WindGustDir'])
  data['RainTomorrow']=le.fit_transform(data['RainTomorrow'])

  #Define the target and variable
  x=data[['MinTemp','MaxTemp','WindGustDir','WindGustSpeed','Humidity','Pressure','Temp']]
  y=data['RainTomorrow']

  #return the feature variable ,target variable and the lable encoder
  return x,y,le

train weather prediction model

In [47]:
def train_model(x,y):
  x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
  model=RandomForestClassifier(n_estimators=100,random_state=42)
  model.fit(x_train,y_train)
  y_pred=model.predict(x_test)
  print("Mean Squared Error For Rain Model")
  print(mean_squared_error(y_test,y_pred))
  return model

Prepare regression data

In [48]:
def prepare_regression_data(data,feature):
  x,y=[],[]

  for i in range(len(data)-1):
    x.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])

  x=np.array(x).reshape(-1,1)
  y=np.array(y)
  return x,y

In [49]:
def train_regression_model(x,y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(x, y)
  return model

prediction

In [50]:
def predict_future(model, current_value):
    predictions = [current_value]

    for i in range(5):
        next_value = model.predict(np.array([[predictions[-1]]]))
        predictions.append(next_value[0])

    return predictions[1:]

In [51]:
# Function to save model
def save_model(model, filename):
    with open(filename, 'wb') as f:
        pickle.dump(model, f)
    print(f"Model saved to {filename}")

# Function to load model
def load_model(filename):
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            model = pickle.load(f)
        print(f"Model loaded from {filename}")
        return model
    else:
        print(f"{filename} not found!")
        return None

weather analysis function

In [53]:
def weather_view():
  city = input("Enter the city name: ")
  current_weather = get_current_weather(city)

  # load historical data
  historical_data = read_historical_data('indian_weather_prediction_dataset.csv')

  # prepare and train the weather prediction model
  x, y, le = prepare_data(historical_data)
  rain_model = train_model(x, y)
  # Check if model already exists
  rain_model = load_model('rain_model.pkl')
  if rain_model is None:
        rain_model = train_model(x, y)
        save_model(rain_model, 'rain_model.pkl')

  # map wind direction to compass point
  wind_deg = current_weather['wind_gust_dir'] % 360
  compass_points = [
      ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
      ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
      ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
      ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
      ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
      ("NNW", 326.25, 348.75), ("N", 348.75, 360)
  ]
  compass_direction = next((point for point, start, end in compass_points if start <= wind_deg < end), "Unknown")
  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

  current_data = {
      'MinTemp': current_weather['temp_min'],
      'MaxTemp': current_weather['temp_max'],
      'WindGustDir': compass_direction_encoded,
      'WindGustSpeed': current_weather.get('wind_speed', 0),
      'Humidity': current_weather['humidity'],
      'Pressure': current_weather['pressure'],
      'Temp': current_weather['current_temp']
  }

  current_df = pd.DataFrame([current_data])

  # weather prediction
  rain_prediction = rain_model.predict(current_df)[0]

  # prepare regression model for temperature and humidity
  x_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
  x_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

  # Check if temperature and humidity models already exist
  temp_model = load_model('temp_model.pkl')
  hum_model = load_model('hum_model.pkl')

  if temp_model is None:
        temp_model = train_regression_model(x_temp, y_temp)
        save_model(temp_model, 'temp_model.pkl')
  if hum_model is None:
        hum_model = train_regression_model(x_hum, y_hum)
        save_model(hum_model, 'hum_model.pkl')


  # prediction future temperature and humidity
  future_temp = predict_future(temp_model, current_weather['temp_min'])
  future_humidity = predict_future(hum_model, current_weather['humidity'])

  timezone = pytz.timezone('Asia/Kolkata')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hours=1)
  next_hour = next_hour.replace(minute=0, second=0, microsecond=0)
  future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

  # display result
  print(f"City: {city}, {current_weather['country']}")
  print(f"Current Temperature: {current_weather['current_temp']}°C")
  print(f"Feels Like: {current_weather['feels_like']}°C")
  print(f"Minimum Temperature: {current_weather['temp_min']}°C")
  print(f"Maximum Temperature: {current_weather['temp_max']}°C")
  print(f"Humidity: {current_weather['humidity']}%")
  print(f"Weather Description: {current_weather['description']}")
  print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

  print("\nFuture Temperature Predictions:")

  for time, temp in zip(future_times, future_temp):
    print(f"{time}: {round(temp, 1)}°C")

  print("\nFuture Humidity Predictions:")

  for time, humidity in zip(future_times, future_humidity):
    print(f"{time}: {round(humidity, 1)}%")

weather_view()


Enter the city name:  Raipur


Mean Squared Error For Rain Model
0.498725
rain_model.pkl not found!
Mean Squared Error For Rain Model
0.498725
Model saved to rain_model.pkl
temp_model.pkl not found!
hum_model.pkl not found!
Model saved to temp_model.pkl
Model saved to hum_model.pkl
City: Raipur, IN
Current Temperature: 21°C
Feels Like: 20°C
Minimum Temperature: 21°C
Maximum Temperature: 21°C
Humidity: 30%
Weather Description: haze
Rain Prediction: No

Future Temperature Predictions:
12:00: 25.0°C
13:00: 25.1°C
14:00: 25.1°C
15:00: 25.1°C
16:00: 25.1°C

Future Humidity Predictions:
12:00: 60.5%
13:00: 60.7%
14:00: 59.5%
15:00: 59.0%
16:00: 60.3%
